# Data Validation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Data Validation

This notebook ensures data quality through various validation checks.
It verifies that the data meets expected schema, statistical properties, and business rules.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Load the dataset

In [ ]:
# For this notebook, we'll assume the data is in a CSV file named 'nz_industry_data.csv'

In [ ]:
# In a real scenario, you would replace this with the actual file path
print("Loading dataset...")
try:
    df = pd.read_csv('nz_industry_data.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Dataset file not found. Creating sample data based on the provided information.")

In [ ]:
    # Create a sample dataset based on the information provided

In [ ]:
    # This is just for demonstration purposes
    years = range(2013, 2024)
    industry_levels = ['Level 1', 'Level 2', 'Level 3', 'Level 4']
    industry_codes = ['99999', 'AA111', 'BB222', 'CC333', 'DD444']
    industry_names = ['All industries', 'Agriculture', 'Manufacturing', 'Services', 'Retail']
    units = ['Dollars (millions)']
    variable_codes = ['H01', 'H04', 'H05', 'H07', 'H08']
    variable_names = ['Total income', 'Sales, government funding, grants and subsidies', 
                      'Interest, dividends and donations', 'Non-operating income', 'Total expenditure']
    variable_categories = ['Financial performance']
    

In [ ]:
    # Create sample data
    data = []
    for year in years:
        for i in range(len(industry_codes)):
            for j in range(len(variable_codes)):
                value = np.random.randint(10000, 1000000)
                anzsic_code = f"ANZSIC06 divisions A-S (excluding classes K6330, L6711)"
                data.append([year, industry_levels[min(i, len(industry_levels)-1)], 
                           industry_codes[i], industry_names[i], units[0], 
                           variable_codes[j], variable_names[j], variable_categories[0], 
                           str(value), anzsic_code])
    
    df = pd.DataFrame(data, columns=['Year', 'Industry_aggregation_NZSIOC', 'Industry_code_NZSIOC',
                                    'Industry_name_NZSIOC', 'Units', 'Variable_code', 'Variable_name',
                                    'Variable_category', 'Value', 'Industry_code_ANZSIC06'])

print("\n")


In [ ]:
# 1. Schema Validation
print("="*80)
print("1. SCHEMA VALIDATION")
print("="*80)


In [ ]:
# 1.1 Check dataframe shape
print(f"DataFrame Shape: {df.shape}")
expected_shape = (50985, 10)
if df.shape == expected_shape:
    print(f"✓ Shape validation passed: {df.shape} matches expected {expected_shape}")
else:
    print(f"✗ Shape validation failed: {df.shape} does not match expected {expected_shape}")


In [ ]:
# 1.2 Check column names
expected_columns = ['Year', 'Industry_aggregation_NZSIOC', 'Industry_code_NZSIOC',
                   'Industry_name_NZSIOC', 'Units', 'Variable_code', 'Variable_name',
                   'Variable_category', 'Value', 'Industry_code_ANZSIC06']

missing_columns = set(expected_columns) - set(df.columns)
extra_columns = set(df.columns) - set(expected_columns)

if len(missing_columns) == 0 and len(extra_columns) == 0:
    print("✓ Column names validation passed: All expected columns are present")
else:
    if len(missing_columns) > 0:
        print(f"✗ Missing columns: {missing_columns}")
    if len(extra_columns) > 0:
        print(f"✗ Extra columns: {extra_columns}")


In [ ]:
# 1.3 Check data types
print("\nColumn Data Types:")
for col in df.columns:
    print(f"{col}: {df[col].dtype}")


In [ ]:
# Check if Year is integer
if df['Year'].dtype == 'int64':
    print("✓ 'Year' column is correctly typed as int64")
else:
    print(f"✗ 'Year' column is {df['Year'].dtype}, expected int64")
    

In [ ]:
# 1.4 Check for null values
null_counts = df.isnull().sum()
print("\nNull Value Counts:")
print(null_counts)

if null_counts.sum() == 0:
    print("✓ No null values found in the dataset")
else:
    print(f"✗ Found {null_counts.sum()} null values in the dataset")

print("\n")


In [ ]:
# 2. Statistical Validation
print("="*80)
print("2. STATISTICAL VALIDATION")
print("="*80)


In [ ]:
# 2.1 Check Year range
year_min = df['Year'].min()
year_max = df['Year'].max()
expected_year_min = 2013
expected_year_max = 2023

print(f"Year range: {year_min} to {year_max}")
if year_min == expected_year_min and year_max == expected_year_max:
    print(f"✓ Year range validation passed: {year_min} to {year_max}")
else:
    print(f"✗ Year range validation failed: Expected {expected_year_min} to {expected_year_max}")


In [ ]:
# 2.2 Convert Value column to numeric
print("\nConverting 'Value' column to numeric...")
df['Value_numeric'] = pd.to_numeric(df['Value'], errors='coerce')


In [ ]:
# Check for conversion failures
if df['Value_numeric'].isna().sum() > 0:
    print(f"✗ {df['Value_numeric'].isna().sum()} values could not be converted to numeric")
else:
    print("✓ All values successfully converted to numeric")


In [ ]:
# 2.3 Check Value distribution
print("\nValue Distribution Statistics:")
value_stats = df['Value_numeric'].describe()
print(value_stats)


In [ ]:
# Check for negative values (which might be invalid for financial data)
neg_values = (df['Value_numeric'] < 0).sum()
if neg_values > 0:
    print(f"✗ Found {neg_values} negative values in 'Value' column")
else:
    print("✓ No negative values found in 'Value' column")


In [ ]:
# 2.4 Check for outliers using IQR method
Q1 = df['Value_numeric'].quantile(0.25)
Q3 = df['Value_numeric'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = ((df['Value_numeric'] < lower_bound) | (df['Value_numeric'] > upper_bound)).sum()
outlier_percentage = (outliers / len(df)) * 100

print(f"\nOutlier Analysis (IQR method):")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")
print(f"Number of outliers: {outliers} ({outlier_percentage:.2f}%)")


In [ ]:
# Visualize Value distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['Value_numeric'], kde=True)
plt.title('Distribution of Value')
plt.xlabel('Value (in millions)')
plt.ylabel('Frequency')
plt.axvline(lower_bound, color='r', linestyle='--', label=f'Lower bound: {lower_bound:.2f}')
plt.axvline(upper_bound, color='r', linestyle='--', label=f'Upper bound: {upper_bound:.2f}')
plt.legend()
plt.show()


In [ ]:
# 2.5 Check distribution of categorical variables
print("\nDistribution of Industry Aggregation Levels:")
industry_level_counts = df['Industry_aggregation_NZSIOC'].value_counts()
print(industry_level_counts)

print("\nDistribution of Variable Categories:")
variable_category_counts = df['Variable_category'].value_counts()
print(variable_category_counts)

print("\nDistribution of Units:")
units_counts = df['Units'].value_counts()
print(units_counts)

print("\n")


In [ ]:
# 3. Business Rule Validation
print("="*80)
print("3. BUSINESS RULE VALIDATION")
print("="*80)


In [ ]:
# 3.1 Check if all years have similar number of records (balanced dataset)
year_counts = df['Year'].value_counts().sort_index()
print("Records per year:")
print(year_counts)

year_count_std = year_counts.std()
year_count_mean = year_counts.mean()
year_count_cv = year_count_std / year_count_mean  # Coefficient of variation

if year_count_cv < 0.1:  # Less than 10% variation
    print(f"✓ Year distribution is balanced (CV: {year_count_cv:.4f})")
else:
    print(f"✗ Year distribution is imbalanced (CV: {year_count_cv:.4f})")


In [ ]:
# 3.2 Check if "All industries" totals are consistent

In [ ]:
# For each year, the "All industries" total income should be greater than or equal to any individual industry
print("\nChecking 'All industries' total income consistency...")


In [ ]:
# Filter for Total income variable
total_income = df[df['Variable_name'] == 'Total income'].copy()


In [ ]:
# Group by Year and check if "All industries" has the highest value
all_industries_check = []
for year, group in total_income.groupby('Year'):
    all_ind = group[group['Industry_name_NZSIOC'] == 'All industries']['Value_numeric'].values
    if len(all_ind) == 0:
        continue
    all_ind_value = all_ind[0]
    max_ind_value = group['Value_numeric'].max()
    
    if all_ind_value >= max_ind_value:
        all_industries_check.append(True)
    else:
        all_industries_check.append(False)
        print(f"✗ In year {year}, 'All industries' value ({all_ind_value}) is not the maximum value ({max_ind_value})")

if all(all_industries_check) and len(all_industries_check) > 0:
    print("✓ 'All industries' total income is consistent across all years")
elif len(all_industries_check) == 0:
    print("! Could not verify 'All industries' consistency - data may be missing")


In [ ]:
# 3.3 Check if Total income >= Total expenditure for each industry and year
print("\nChecking income >= expenditure business rule...")


In [ ]:
# Get income and expenditure data
income_exp_check = []
for (year, industry), group in df[df['Variable_name'].isin(['Total income', 'Total expenditure'])].groupby(['Year', 'Industry_name_NZSIOC']):
    if len(group) < 2:
        continue
        
    income = group[group['Variable_name'] == 'Total income']['Value_numeric'].values
    expenditure = group[group['Variable_name'] == 'Total expenditure']['Value_numeric'].values
    
    if len(income) == 0 or len(expenditure) == 0:
        continue
        
    income_value = income[0]
    expenditure_value = expenditure[0]
    

In [ ]:
    # Allow for small differences due to rounding or other factors
    if income_value >= expenditure_value * 0.5:  # Income should be at least 50% of expenditure as a loose rule
        income_exp_check.append(True)
    else:
        income_exp_check.append(False)
        print(f"✗ In year {year}, industry '{industry}' has income ({income_value}) < 50% of expenditure ({expenditure_value})")

if all(income_exp_check) and len(income_exp_check) > 0:
    print("✓ Income and expenditure relationships are reasonable")
elif len(income_exp_check) == 0:
    print("! Could not verify income/expenditure relationship - data may be missing")


In [ ]:
# 3.4 Check if industry codes match industry names consistently
print("\nChecking industry code-name consistency...")

industry_code_name_map = df[['Industry_code_NZSIOC', 'Industry_name_NZSIOC']].drop_duplicates()
industry_code_counts = industry_code_name_map['Industry_code_NZSIOC'].value_counts()
industry_name_counts = industry_code_name_map['Industry_name_NZSIOC'].value_counts()

inconsistent_codes = industry_code_counts[industry_code_counts > 1]
inconsistent_names = industry_name_counts[industry_name_counts > 1]

if len(inconsistent_codes) == 0:
    print("✓ Each industry code maps to exactly one industry name")
else:
    print(f"✗ Found {len(inconsistent_codes)} industry codes that map to multiple industry names")
    for code in inconsistent_codes.index:
        names = industry_code_name_map[industry_code_name_map['Industry_code_NZSIOC'] == code]['Industry_name_NZSIOC'].unique()
        print(f"  Code {code} maps to: {names}")

if len(inconsistent_names) == 0:
    print("✓ Each industry name maps to exactly one industry code")
else:
    print(f"✗ Found {len(inconsistent_names)} industry names that map to multiple industry codes")
    for name in inconsistent_names.index:
        codes = industry_code_name_map[industry_code_name_map['Industry_name_NZSIOC'] == name]['Industry_code_NZSIOC'].unique()
        print(f"  Name '{name}' maps to codes: {codes}")

print("\n")


In [ ]:
# 4. Data Quality Checks
print("="*80)
print("4. DATA QUALITY CHECKS")
print("="*80)


In [ ]:
# 4.1 Check for duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")
if duplicate_count == 0:
    print("✓ No duplicate rows found")
else:
    print(f"✗ Found {duplicate_count} duplicate rows")


In [ ]:
# 4.2 Check for consistency in units
unique_units = df['Units'].unique()
print(f"\nUnique units in the dataset: {unique_units}")
if len(unique_units) == 1 and 'Dollars (millions)' in unique_units:
    print("✓ Units are consistent: 'Dollars (millions)'")
else:
    print(f"✗ Units are inconsistent: found {len(unique_units)} different units")


In [ ]:
# 4.3 Check for consistency in variable categories
unique_categories = df['Variable_category'].unique()
print(f"\nUnique variable categories: {unique_categories}")
if len(unique_categories) <= 3:  # Allowing for a reasonable number of categories
    print(f"✓ Variable categories are consistent: {unique_categories}")
else:
    print(f"✗ Too many variable categories: {len(unique_categories)}")


In [ ]:
# 4.4 Check for unusual patterns in the data

In [ ]:
# Look for industries with extreme changes year over year
print("\nChecking for unusual year-over-year changes...")


In [ ]:
# Focus on Total income for this check
total_income = df[df['Variable_name'] == 'Total income'].copy()
if len(total_income) > 0:

In [ ]:
    # Calculate year-over-year change for each industry
    yoy_changes = []
    for industry, group in total_income.groupby('Industry_name_NZSIOC'):
        group = group.sort_values('Year')
        if len(group) < 2:
            continue
            
        group['prev_value'] = group['Value_numeric'].shift(1)
        group['yoy_change'] = (group['Value_numeric'] - group['prev_value']) / group['prev_value']
        group = group.dropna()
        

In [ ]:
        # Find extreme changes (more than 100% increase or 50% decrease)
        extreme_changes = group[(group['yoy_change'] > 1.0) | (group['yoy_change'] < -0.5)]
        
        for _, row in extreme_changes.iterrows():
            yoy_changes.append({
                'Industry': industry,
                'Year': row['Year'],
                'Previous Value': row['prev_value'],
                'Current Value': row['Value_numeric'],
                'Change %': row['yoy_change'] * 100
            })
    
    if len(yoy_changes) > 0:
        print(f"✗ Found {len(yoy_changes)} instances of extreme year-over-year changes:")
        yoy_df = pd.DataFrame(yoy_changes)
        print(yoy_df.head(10))  # Show first 10 extreme changes
    else:
        print("✓ No extreme year-over-year changes detected")
else:
    print("! Could not check year-over-year changes - 'Total income' data may be missing")


In [ ]:
# 4.5 Check for consistency in ANZSIC06 codes
print("\nChecking ANZSIC06 code consistency...")


In [ ]:
# Extract the main part of the ANZSIC06 code (before any exclusions)
df['ANZSIC06_main'] = df['Industry_code_ANZSIC06'].str.extract(r'(ANZSIC06 [^(]+)')

anzsic_counts = df['ANZSIC06_main'].value_counts()
if len(anzsic_counts) <= 10:  # Allowing for a reasonable number of main ANZSIC patterns
    print(f"✓ ANZSIC06 codes follow consistent patterns ({len(anzsic_counts)} main patterns)")
else:
    print(f"✗ ANZSIC06 codes have too many patterns ({len(anzsic_counts)} main patterns)")

print("\n")


In [ ]:
# 5. Reporting Validation Results
print("="*80)
print("5. VALIDATION RESULTS SUMMARY")
print("="*80)


In [ ]:
# Create a summary of validation results
validation_results = {
    'Schema Validation': {
        'Shape Check': df.shape == expected_shape,
        'Column Names': len(missing_columns) == 0 and len(extra_columns) == 0,
        'Data Types': df['Year'].dtype == 'int64',
        'Null Values': null_counts.sum() == 0
    },
    'Statistical Validation': {
        'Year Range': year_min == expected_year_min and year_max == expected_year_max,
        'Value Conversion': df['Value_numeric'].isna().sum() == 0,
        'Negative Values': neg_values == 0,
        'Outliers': outlier_percentage < 5.0  # Less than 5% outliers is acceptable
    },
    'Business Rule Validation': {
        'Balanced Years': year_count_cv < 0.1,
        'All Industries Consistency': all(all_industries_check) if len(all_industries_check) > 0 else None,
        'Income-Expenditure Relation': all(income_exp_check) if len(income_exp_check) > 0 else None,
        'Code-Name Consistency': len(inconsistent_codes) == 0 and len(inconsistent_names) == 0
    },
    'Data Quality': {
        'No Duplicates': duplicate_count == 0,
        'Consistent Units': len(unique_units) == 1 and 'Dollars (millions)' in unique_units,
        'Consistent Categories': len(unique_categories) <= 3,
        'No Extreme Changes': len(yoy_changes) == 0 if 'yoy_changes' in locals() else None,
        'ANZSIC06 Consistency': len(anzsic_counts) <= 10
    }
}


In [ ]:
# Convert to DataFrame for better visualization
results_list = []
for category, checks in validation_results.items():
    for check_name, result in checks.items():
        status = "PASS" if result == True else "FAIL" if result == False else "UNKNOWN"
        results_list.append({
            'Category': category,
            'Check': check_name,
            'Status': status
        })

results_df = pd.DataFrame(results_list)


In [ ]:
# Print summary
print("Validation Results Summary:")
print(results_df)


In [ ]:
# Calculate pass rate
pass_count = (results_df['Status'] == 'PASS').sum()
total_checks = len(results_df[results_df['Status'] != 'UNKNOWN'])
pass_rate = (pass_count / total_checks) * 100 if total_checks > 0 else 0

print(f"\nOverall Pass Rate: {pass_rate:.2f}% ({pass_count}/{total_checks} checks passed)")


In [ ]:
# Generate recommendations based on failed checks
failed_checks = results_df[results_df['Status'] == 'FAIL']
if len(failed_checks) > 0:
    print("\nRecommendations for Failed Checks:")
    for _, row in failed_checks.iterrows():
        category = row['Category']
        check = row['Check']
        
        if category == 'Schema Validation':
            if check == 'Shape Check':
                print("- Verify the dataset is complete and has the expected number of records")
            elif check == 'Column Names':
                print("- Check for missing or extra columns and align with expected schema")
            elif check == 'Data Types':
                print("- Convert Year column to integer type")
            elif check == 'Null Values':
                print("- Handle or investigate null values in the dataset")
                
        elif category == 'Statistical Validation':
            if check == 'Year Range':
                print("- Verify the dataset covers the expected time period")
            elif check == 'Value Conversion':
                print("- Clean the Value column to ensure all entries can be converted to numeric")
            elif check == 'Negative Values':
                print("- Investigate negative values in financial data")
            elif check == 'Outliers':
                print("- Review outliers in the Value column")
                
        elif category == 'Business Rule Validation':
            if check == 'Balanced Years':
                print("- Investigate why some years have significantly different record counts")
            elif check == 'All Industries Consistency':
                print("- Check why 'All industries' totals are inconsistent with individual industries")
            elif check == 'Income-Expenditure Relation':
                print("- Review cases where income is significantly less than expenditure")
            elif check == 'Code-Name Consistency':
                print("- Standardize industry code to name mappings")
                
        elif category == 'Data Quality':
            if check == 'No Duplicates':
                print("- Remove duplicate records from the dataset")
            elif check == 'Consistent Units':
                print("- Standardize units across the dataset")
            elif check == 'Consistent Categories':
                print("- Review and consolidate variable categories")
            elif check == 'No Extreme Changes':
                print("- Investigate industries with extreme year-over-year changes")
            elif check == 'ANZSIC06 Consistency':
                print("- Standardize ANZSIC06 code formats")


In [ ]:
# Save validation results to CSV
results_df.to_csv('validation_results.csv', index=False)
print("\nValidation results saved to 'validation_results.csv'")


In [ ]:
# Create a visualization of validation results
plt.figure(figsize=(10, 6))
status_counts = results_df['Status'].value_counts()
colors = {'PASS': 'green', 'FAIL': 'red', 'UNKNOWN': 'gray'}
status_counts.plot(kind='bar', color=[colors[x] for x in status_counts.index])
plt.title('Validation Results Summary')
plt.xlabel('Status')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# Create a visualization of validation results by category
plt.figure(figsize=(12, 8))
category_results = results_df.pivot_table(
    index='Category', 
    columns='Status', 
    aggfunc='size', 
    fill_value=0
)

category_results.plot(kind='bar', stacked=True, color=[colors[x] for x in category_results.columns])
plt.title('Validation Results by Category')
plt.xlabel('Category')
plt.ylabel('Count')
plt.legend(title='Status')
plt.tight_layout()
plt.show()

print("\nData validation complete.")